In [331]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime

In [332]:
# Import banking data
# Note: This data was extracted on 1999

# Each record describes characteristics of a client
client = pd.read_csv('./data/raw/data_berka/client.asc', sep=';')

# Each record describes static characteristics of an account
account = pd.read_csv('./data/raw/data_berka/account.asc', sep=';')

# Each record describes a credit card issued to an account
card = pd.read_csv('./data/raw/data_berka/card.asc', sep=';')

# Each record describes demographic characteristics of a district
district = pd.read_csv('./data/raw/data_berka/district.asc', sep=';')

# Each record relates together a client with an account
# i.e. this relation describes the rights of clients to operate accounts
disp = pd.read_csv('./data/raw/data_berka/disp.asc', sep=';')

# Each record describes characteristics of a payment order (debits only)
order = pd.read_csv('./data/raw/data_berka/order.asc', sep=';')

# Each record describes one transaction on an account
trans = pd.read_csv('./data/raw/data_berka/trans.asc', sep=';', low_memory=False)

# Each record describes a loan granted for a given account
loan = pd.read_csv('./data/raw/data_berka/loan.asc', sep=';')

In [333]:
def checkEmpty(df):
    empty = {}
    for column in df.columns[0:]:
            empty[column] = df[column].astype(str).str.contains(r'^\s*$', na=False)
    return empty 

In [334]:
def fillEmpty(df, empty):
    for column in df.columns[0:]:
        if df[column].dtype == object:
            df.loc[emptyrows[column], column] = 'Other'
    return df

In [335]:
def renameColumn(table, df):
    for column in df.columns[1:]:
        if table not in column:
            column_name =  table + '_' + column
            df.rename(columns={column : column_name}, inplace = True)
    return df

In [336]:
client = renameColumn('client', client)
# Checking missing columns
client['client_birth_year'] = client['client_birth_number'].astype(str).str[:2].astype(int)
client['client_birth_year'] = client['client_birth_year'] + 1900
#calculate the age of clients
client['client_age'] = 1997 - client['client_birth_year']
#age group for clients
client['client_age_group'] = client['client_age'] // 10 * 10

client['client_birth_month'] = client['client_birth_number'].astype(str).str[2:4].astype(int)
client['client_gender'] = 'M'
client.loc[client['client_birth_month'] > 50, 'client_gender'] = 'F'
client.loc[client['client_birth_month'] > 50, 'client_birth_month'] = client['client_birth_month'] - 50
client['client_birth_day'] = client['client_birth_number'].astype(str).str[4:6].astype(int)

client["client_category"]=np.where(client['client_age_group'] < 21 ,"Young(< 21)",
                                     (np.where(client['client_age_group'] < 55 , "Adult(21-55)" ,"Senior Citizen(> 55)")))

# Checking empty columns
emptyrows = checkEmpty(client)
client =  fillEmpty(client, emptyrows)

In [337]:
account = renameColumn('account', account)
# Checking missing columns
print("The number of missing columns are: " +  str(account.isna().sum().sum()))
#Processing Date
account['account_year'] = account['account_date'].astype(str).str[:2].astype(int) 
account['account_year'] = account['account_year'] + 1900
account['account_month'] = account['account_date'].astype(str).str[2:4].astype(int)
account['account_day'] = account['account_date'].astype(str).str[4:6].astype(int)

#getting Length of relationship (LOR) of the accounts
account['account_lor'] = 1997 - account['account_year']



# Checking empty columns
emptyrows = checkEmpty(account)
account =  fillEmpty(account, emptyrows)
account.info()

The number of missing columns are: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   account_id           4500 non-null   int64 
 1   account_district_id  4500 non-null   int64 
 2   account_frequency    4500 non-null   object
 3   account_date         4500 non-null   int64 
 4   account_year         4500 non-null   int32 
 5   account_month        4500 non-null   int32 
 6   account_day          4500 non-null   int32 
 7   account_lor          4500 non-null   int32 
dtypes: int32(4), int64(3), object(1)
memory usage: 211.1+ KB


In [338]:
#Rename Columns
disp = renameColumn('disp', disp)
# Checking missing columns
print("The number of missing columns are: " +  str(disp.isna().sum().sum()))
# Checking empty columns
emptyrows = checkEmpty(disp)
disp =  fillEmpty(disp, emptyrows)

The number of missing columns are: 0


In [339]:
account_disp_merge = account.merge(disp, left_on = 'account_id', right_on = 'disp_account_id', how = 'left')

In [340]:
client_disp_merge = account_disp_merge.merge(client, left_on = 'disp_client_id', right_on = 'client_id', how = 'left')
client_disp_merge.loc[client_disp_merge['disp_type'] == 'DISPONENT','acc_has_disp'] = 1
client_disp_merge['acc_has_disp'] = client_disp_merge['acc_has_disp'].fillna(0)

#Check for accounts with more than one owner
summary = client_disp_merge[client_disp_merge['disp_type'] == 'OWNER'].groupby('disp_account_id')['disp_client_id'].count()
summary.unique()

#There are no accounts with more than one owner

array([1], dtype=int64)

In [341]:
account_client_count = client_disp_merge.groupby(['disp_account_id'])[['acc_has_disp']].agg('sum')
client_disp_merge = account_client_count.merge(client_disp_merge, left_on = 'disp_account_id', right_on = 'disp_account_id', how = 'left')
client_disp_merge = client_disp_merge[client_disp_merge['disp_type'] == 'OWNER']
client_disp_merge.drop(['disp_client_id', 'client_birth_number', 'acc_has_disp_x'], axis=1, inplace=True)
client_disp_merge.rename(columns={'acc_has_disp_y' : 'num_disp'}, inplace = True)
client_disp_merge.head()

,disp_account_id,account_id,account_district_id,account_frequency,account_date,account_year,account_month,account_day,account_lor,disp_id,...,client_id,client_district_id,client_birth_year,client_age,client_age_group,client_birth_month,client_gender,client_birth_day,client_category,num_disp
0,1,1,18,POPLATEK MESICNE,950324,1995,3,24,2,1,...,1,18,1970,27,20,12,F,13,Young(< 21),0.0
1,2,2,1,POPLATEK MESICNE,930226,1993,2,26,4,2,...,2,1,1945,52,50,2,M,4,Adult(21-55),0.0
3,3,3,5,POPLATEK MESICNE,970707,1997,7,7,0,4,...,4,5,1956,41,40,12,M,1,Adult(21-55),0.0
5,4,4,12,POPLATEK MESICNE,960221,1996,2,21,1,6,...,6,12,1919,78,70,9,M,22,Senior Citizen(> 55),0.0
6,5,5,15,POPLATEK MESICNE,970530,1997,5,30,0,7,...,7,15,1929,68,60,1,M,25,Senior Citizen(> 55),0.0


In [342]:
#Replacing '?' in A12, A15 column with 0.0 to remove impact
district.loc[district['A12'] == '?', 'A12'] = 0.0
district.loc[district['A15'] == '?', 'A15'] = 0
#Changing to column to Float, int
district['A12'] = district['A12'].astype(float)
district['A15'] = district['A15'].astype(int)

#Renaming district columns
district.rename(columns={'A1' : 'District_Code'}, inplace = True)
district.rename(columns={'A2' : 'District_Name'}, inplace = True)
district.rename(columns={'A3' : 'Region'}, inplace = True)
district.rename(columns={'A4' : 'n_inhabitants'}, inplace = True)
district.rename(columns={'A5' : 'municipalities_inh_499'}, inplace = True)
district.rename(columns={'A6' : 'municipalities_500_1999'}, inplace = True)
district.rename(columns={'A7' : 'municipalities_2000_9999'}, inplace = True)
district.rename(columns={'A8' : 'municipalities_10000'}, inplace = True)
district.rename(columns={'A9' : 'n_cities'}, inplace = True)
district.rename(columns={'A10' : 'ratio_urban_inh'}, inplace = True)
district.rename(columns={'A11' : 'average_salary'}, inplace = True)
district.rename(columns={'A12' : 'unemploymant_rate_95'}, inplace = True)
district.rename(columns={'A13' : 'unemploymant_rate_96'}, inplace = True)
district.rename(columns={'A14' : 'entrepreneurs_per_1000'}, inplace = True)
district.rename(columns={'A15' : 'committed_crimes_95'}, inplace = True)
district.rename(columns={'A16' : 'committed_crimes_96'}, inplace = True)

# Checking empty columns
emptyrows = checkEmpty(district)
district =  fillEmpty(district, emptyrows)
district.head()

,District_Code,District_Name,Region,n_inhabitants,municipalities_inh_499,municipalities_500_1999,municipalities_2000_9999,municipalities_10000,n_cities,ratio_urban_inh,average_salary,unemploymant_rate_95,unemploymant_rate_96,entrepreneurs_per_1000,committed_crimes_95,committed_crimes_96
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.67,1.85,132,2159,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,2824,2813
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,5244,5892
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.85,4.43,118,2616,3040


In [343]:
district.loc[district['unemploymant_rate_96'] > district['unemploymant_rate_95'], 'increase_in_unemploymant_rate'] = district['unemploymant_rate_96'] - district['unemploymant_rate_95']
district.loc[district['committed_crimes_96'] > district['committed_crimes_95'], 'increase_in_committed_crimes'] = district['committed_crimes_96'] - district['committed_crimes_95']
district.loc[district['unemploymant_rate_96'] < district['unemploymant_rate_95'], 'decrease_in_unemploymant_rate'] = district['unemploymant_rate_95'] - district['unemploymant_rate_96']
district.loc[district['committed_crimes_96'] < district['committed_crimes_95'], 'decrease_in_committed_crimes'] = district['committed_crimes_95'] - district['committed_crimes_96']
district['increase_in_unemploymant_rate'] = district['increase_in_unemploymant_rate'].fillna(0)
district['increase_in_committed_crimes'] = district['increase_in_committed_crimes'].fillna(0)
district['decrease_in_unemploymant_rate'] = district['decrease_in_unemploymant_rate'].fillna(0)
district['decrease_in_committed_crimes'] = district['decrease_in_committed_crimes'].fillna(0)

district.loc[:,'total_municipalities'] = district['municipalities_inh_499'] + district['municipalities_500_1999'] + district['municipalities_2000_9999'] + district['municipalities_10000']
district.loc[:, 'number_of_employed_population_96'] =  district['n_inhabitants'] - district['unemploymant_rate_96']/100 * district['n_inhabitants']
district.loc[:, 'number_of_employed_urban_population_96'] =  district['number_of_employed_population_96'] - district['number_of_employed_population_96'] / district['ratio_urban_inh']
district['number_of_employed_population_96'] = district['number_of_employed_population_96'].astype(int)
district['number_of_employed_urban_population_96'] = district['number_of_employed_urban_population_96'].astype(int)
district.head()

,District_Code,District_Name,Region,n_inhabitants,municipalities_inh_499,municipalities_500_1999,municipalities_2000_9999,municipalities_10000,n_cities,ratio_urban_inh,...,entrepreneurs_per_1000,committed_crimes_95,committed_crimes_96,increase_in_unemploymant_rate,increase_in_committed_crimes,decrease_in_unemploymant_rate,decrease_in_committed_crimes,total_municipalities,number_of_employed_population_96,number_of_employed_urban_population_96
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,...,167,85677,99107,0.14,13430.0,0.0,0.0,1,1199771,1187773
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,...,132,2159,2674,0.18,515.0,0.0,0.0,114,87239,85371
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,...,111,2824,2813,0.26,0.0,0.0,11.0,86,73569,71805
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,...,109,5244,5892,0.41,648.0,0.0,0.0,100,142323,140211
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,...,118,2616,3040,0.58,424.0,0.0,0.0,100,91380,89602


In [344]:
client_disp_dist_merge = client_disp_merge.merge(district, left_on = 'client_district_id', right_on='District_Code', how = 'left' )
client_disp_merge.drop(['account_district_id', 'account_date', 'disp_account_id', 'account_date','client_district_id','disp_account_id'], axis=1, inplace=True)
client_disp_merge.head()

,account_id,account_frequency,account_year,account_month,account_day,account_lor,disp_id,disp_type,client_id,client_birth_year,client_age,client_age_group,client_birth_month,client_gender,client_birth_day,client_category,num_disp
0,1,POPLATEK MESICNE,1995,3,24,2,1,OWNER,1,1970,27,20,12,F,13,Young(< 21),0.0
1,2,POPLATEK MESICNE,1993,2,26,4,2,OWNER,2,1945,52,50,2,M,4,Adult(21-55),0.0
3,3,POPLATEK MESICNE,1997,7,7,0,4,OWNER,4,1956,41,40,12,M,1,Adult(21-55),0.0
5,4,POPLATEK MESICNE,1996,2,21,1,6,OWNER,6,1919,78,70,9,M,22,Senior Citizen(> 55),0.0
6,5,POPLATEK MESICNE,1997,5,30,0,7,OWNER,7,1929,68,60,1,M,25,Senior Citizen(> 55),0.0


In [345]:
#Rename Columns
order = renameColumn('order', order)
# Checking missing columns
print("The number of missing columns are: " +  str(order.isna().sum().sum()))
# Checking empty columns
emptyrows = checkEmpty(order)
order =  fillEmpty(order, emptyrows)
order.head()


The number of missing columns are: 0


,order_id,order_account_id,order_bank_to,order_account_to,order_amount,order_k_symbol
0,29401,1,YZ,87144583,2452.0,SIPO
1,29402,2,ST,89597016,3372.7,UVER
2,29403,2,QR,13943797,7266.0,SIPO
3,29404,3,WX,83084338,1135.0,SIPO
4,29405,3,CD,24485939,327.0,Other


In [346]:
order_freq1 = order.groupby(by=['order_account_id'], as_index=True)['order_amount'].agg(["count","sum"])
order_ksymbol_freq = order.groupby(by=['order_account_id', 'order_k_symbol'])['order_amount'].sum().unstack('order_k_symbol', fill_value=0)
order_freq1.head()




,count,sum
order_account_id,,
1,1,2452.0
2,2,10638.7
3,3,5001.0
4,2,3363.0
5,1,2668.0


In [347]:
order_ksymbol_freq.head()

order_k_symbol,LEASING,Other,POJISTNE,SIPO,UVER
order_account_id,,,,,
1,0.0,0.0,0.0,2452.0,0.0
2,0.0,0.0,0.0,7266.0,3372.7
3,0.0,327.0,3539.0,1135.0,0.0
4,0.0,0.0,0.0,3363.0,0.0
5,0.0,0.0,0.0,2668.0,0.0


In [348]:
processed_table = client_disp_dist_merge.merge(order_freq1, left_on = 'account_id', right_on = 'order_account_id', how = 'left')
processed_table = processed_table.merge(order_ksymbol_freq, left_on = 'account_id', right_on = 'order_account_id', how = 'left')

In [349]:
processed_table.rename(columns={'count' : 'num_order','sum' : 'total_order_amount' }, inplace = True)
processed_table.head()

,disp_account_id,account_id,account_district_id,account_frequency,account_date,account_year,account_month,account_day,account_lor,disp_id,...,total_municipalities,number_of_employed_population_96,number_of_employed_urban_population_96,num_order,total_order_amount,LEASING,Other,POJISTNE,SIPO,UVER
0,1,1,18,POPLATEK MESICNE,950324,1995,3,24,2,1,...,76,68330,67284,1.0,2452.0,0.0,0.0,0.0,2452.0,0.0
1,2,2,1,POPLATEK MESICNE,930226,1993,2,26,4,2,...,1,1199771,1187773,2.0,10638.7,0.0,0.0,0.0,7266.0,3372.7
2,3,3,5,POPLATEK MESICNE,970707,1997,7,7,0,4,...,100,91380,89602,3.0,5001.0,0.0,327.0,3539.0,1135.0,0.0
3,4,4,12,POPLATEK MESICNE,960221,1996,2,21,1,6,...,120,103220,101441,2.0,3363.0,0.0,0.0,0.0,3363.0,0.0
4,5,5,15,POPLATEK MESICNE,970530,1997,5,30,0,7,...,46,56679,55587,1.0,2668.0,0.0,0.0,0.0,2668.0,0.0


In [350]:
#Rename Columns
loan = renameColumn('loan', loan)
# Checking missing columns
print("The number of missing columns are: " +  str(loan.isna().sum().sum()))


#Processing Date
loan['loan_year'] = loan['loan_date'].astype(str).str[:2].astype(int) 
loan['loan_year'] = loan['loan_year'] + 1900
loan['loan_month'] = loan['loan_date'].astype(str).str[2:4].astype(int)
loan['loan_day'] = loan['loan_date'].astype(str).str[4:6].astype(int)



#Current Loan Status
loan["cur_loan_Status"] = 'Not Available'
loan.loc[loan["loan_status"]=="A","cur_loan_Status"]="Closed"
loan.loc[loan["loan_status"]=="B","cur_loan_Status"]="Closed - Defaulted"
loan.loc[loan["loan_status"]=="C","cur_loan_Status"]="Running"
loan.loc[loan["loan_status"]=="D","cur_loan_Status"]="Running - in Debt"

loan["loan_category"]=np.where(loan['loan_amount'] < 196940 ,"Low",
                                     (np.where(loan['loan_amount'] < 393880.0 , "Medium" ,"High")))

 

#creating target Variable 
loan.loc[loan["loan_year"] == 1997,'loan_granted'] = 1

# Checking empty columns
emptyrows = checkEmpty(loan)
loan =  fillEmpty(loan, emptyrows)


The number of missing columns are: 0


In [351]:
loan.head()

,loan_id,loan_account_id,loan_date,loan_amount,loan_duration,loan_payments,loan_status,loan_year,loan_month,loan_day,cur_loan_Status,loan_category,loan_granted
0,5314,1787,930705,96396,12,8033.0,B,1993,7,5,Closed - Defaulted,Low,NaN
1,5316,1801,930711,165960,36,4610.0,A,1993,7,11,Closed,Low,NaN
2,6863,9188,930728,127080,60,2118.0,A,1993,7,28,Closed,Low,NaN
3,5325,1843,930803,105804,36,2939.0,A,1993,8,3,Closed,Low,NaN
4,7240,11013,930906,274740,60,4579.0,A,1993,9,6,Closed,Medium,NaN


In [352]:
loan = loan.groupby('loan_account_id').sum()
processed_table = processed_table.merge(loan, left_on = 'account_id', right_on = 'loan_account_id', how = 'left')
processed_table.head()

,disp_account_id,account_id,account_district_id,account_frequency,account_date,account_year,account_month,account_day,account_lor,disp_id,...,UVER,loan_id,loan_date,loan_amount,loan_duration,loan_payments,loan_year,loan_month,loan_day,loan_granted
0,1,1,18,POPLATEK MESICNE,950324,1995,3,24,2,1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2,1,POPLATEK MESICNE,930226,1993,2,26,4,2,...,3372.7,4959.0,940105.0,80952.0,24.0,3373.0,1994.0,1.0,5.0,0.0
2,3,3,5,POPLATEK MESICNE,970707,1997,7,7,0,4,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,4,12,POPLATEK MESICNE,960221,1996,2,21,1,6,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,5,15,POPLATEK MESICNE,970530,1997,5,30,0,7,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [353]:
#Rename Columns
card = renameColumn('card', card)
# Checking missing columns
print("The number of missing columns are: " +  str(card.isna().sum().sum()))
#Processing Date
card['card_issued_year'] = card['card_issued'].astype(str).str[:2].astype(int) 
card['card_issued_year'] = card['card_issued_year'] + 1900
card['card_issued_month'] = card['card_issued'].astype(str).str[2:4].astype(int)
card['card_issued_day'] = card['card_issued'].astype(str).str[4:6].astype(int)

#change the date format
card['card_date_issued'] = pd.to_datetime(card['card_issued']).dt.date


card["card_LOR"] = datetime.datetime(1997,1,1) - (pd.to_datetime(card.card_date_issued))

#removing negative values for LOR
card['card_LOR'] = abs(card['card_LOR'])

#creating target Variable 
card.loc[card["card_issued_year"] == 1997,'card_issued_97'] = 1



# Checking empty columns
emptyrows = checkEmpty(card)
card =  fillEmpty(card, emptyrows)

The number of missing columns are: 0


In [354]:
card.head()

,card_id,card_disp_id,card_type,card_issued,card_issued_year,card_issued_month,card_issued_day,card_date_issued,card_LOR,card_issued_97
0,1005,9285,classic,931107 00:00:00,1993,11,7,1993-11-07,1151 days,NaN
1,104,588,classic,940119 00:00:00,1994,1,19,1994-01-19,1078 days,NaN
2,747,4915,classic,940205 00:00:00,1994,2,5,1994-02-05,1061 days,NaN
3,70,439,classic,940208 00:00:00,1994,2,8,1994-02-08,1058 days,NaN
4,577,3687,classic,940215 00:00:00,1994,2,15,1994-02-15,1051 days,NaN


In [355]:
card_disp_merge = disp.merge(card, left_on = 'disp_id', right_on = 'card_disp_id', how = 'inner')
summary = card_disp_merge.groupby('disp_account_id')['card_id'].agg('count')
summary.unique()
#All accounts have have one card assoiciated with the account

array([1], dtype=int64)

In [356]:
processed_table = processed_table.merge(card_disp_merge, left_on = 'account_id', right_on = 'disp_account_id', how = 'left')
processed_table.drop(['disp_account_id_x', 'disp_id_x', 'disp_type_x', 'disp_id_y', 'disp_account_id_y', 'disp_type_y', 'card_disp_id'], axis=1, inplace=True)
processed_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4500 entries, 0 to 4499
Data columns (total 67 columns):
 #   Column                                  Non-Null Count  Dtype          
---  ------                                  --------------  -----          
 0   account_id                              4500 non-null   int64          
 1   account_district_id                     4500 non-null   int64          
 2   account_frequency                       4500 non-null   object         
 3   account_date                            4500 non-null   int64          
 4   account_year                            4500 non-null   int32          
 5   account_month                           4500 non-null   int32          
 6   account_day                             4500 non-null   int32          
 7   account_lor                             4500 non-null   int32          
 8   client_id                               4500 non-null   int64          
 9   client_district_id                      4

In [357]:
#Rename Columns
trans = renameColumn('trans', trans)
# Checking missing columns
print("The number of missing columns are: " +  str(trans.isna().sum().sum()))

emptyrows = checkEmpty(trans)
#check na per column 
for column in emptyrows:
    print("the empty rows in " + column + " are: ")
    print(trans[column].isna().sum())

trans =  fillEmpty(trans, emptyrows)

The number of missing columns are: 2208738
the empty rows in trans_id are: 
0
the empty rows in trans_account_id are: 
0
the empty rows in trans_date are: 
0
the empty rows in trans_type are: 
0
the empty rows in trans_operation are: 
183114
the empty rows in trans_amount are: 
0
the empty rows in trans_balance are: 
0
the empty rows in trans_k_symbol are: 
481881
the empty rows in trans_bank are: 
782812
the empty rows in trans_account are: 
760931


In [358]:
#Rename Columns 
trans.rename(columns={'k_symbol' : 'trans_char'}, inplace = True)
trans.rename(columns={'bank' : 'partner_bank'}, inplace = True)
trans.rename(columns={'account' : 'partner_account'}, inplace = True)
trans.rename(columns={'operation' : 'trans_mode'}, inplace = True)
trans.rename(columns={'type' : 'trans_type'}, inplace = True)
trans.rename(columns={'date' : 'trans_date'}, inplace = True)


# Extract the transaction day
trans['trans_day'] = trans['trans_date'].astype(str).str[-2:].astype(int)

# Extract the transaction month
trans['trans_month'] = trans['trans_date'].astype(str).str[2:4].astype(int)

#Extract the transaction year
trans['trans_year'] = trans['trans_date'].astype(str).str[:2].astype(int) + 1900


#changing the trans_date format
trans['trans_date'] = '19' + trans['trans_date'].astype(str)
trans['trans_date'] = pd.to_datetime(trans['trans_date'], format='%Y%m%d').dt.strftime("%d-%m-%Y")

In [359]:
trans.rename(columns={'trans_k_symbol' : 'trans_char'}, inplace = True)
trans.rename(columns={'trans_bank' : 'partner_bank'}, inplace = True)
trans.rename(columns={'trans_account' : 'partner_account'}, inplace = True)
trans.rename(columns={'trans_operation' : 'trans_mode'}, inplace = True)
trans.rename(columns={'type' : 'trans_type'}, inplace = True)
trans.rename(columns={'date' : 'trans_date'}, inplace = True)

In [360]:
trans.rename(columns={'trans_account_id' : 'account_id'}, inplace = True)

In [361]:
trans['date'] = pd.DatetimeIndex(trans['trans_date']).date

In [362]:
# Calculating RECENCY of the transactions


recency_df = trans.groupby(by='account_id', as_index=False)['date'].max()

#naming extracted last_trans_date column
recency_df.columns = ['account_id','last_trans_date']

recent_date=recency_df.last_trans_date.max()

In [363]:
#Calculating recency from the last date
recency_df['trans_recency'] = recency_df['last_trans_date'].apply(lambda x: (recent_date - x).days)
recency_df.head(1)

,account_id,last_trans_date,trans_recency
0,1,1998-12-31,0


In [364]:
# Calculating FREQUENCY (ow often a tansaction is made) of the transaction

# Frequency - How often a transaction is made

freq_df = trans.groupby(by=['account_id'], as_index=False)['trans_id'].count()
freq_df.columns = ['account_id','trans_frequency']
freq_df.head(1)

,account_id,trans_frequency
0,1,239


In [366]:
processed_table =  processed_table.merge(freq_df, left_on = 'account_id', right_on = 'account_id', how = 'left')

In [367]:
monetary_df = trans.groupby(by='account_id',as_index=False).agg({'trans_amount': 'sum'})
monetary_df.columns = ['account_id','monetary']
monetary_df.head(1)

,account_id,monetary
0,1,375174.5


In [368]:
processed_table =  processed_table.merge(monetary_df, left_on = 'account_id', right_on = 'account_id', how = 'left')

In [369]:
processed_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4500 entries, 0 to 4499
Data columns (total 69 columns):
 #   Column                                  Non-Null Count  Dtype          
---  ------                                  --------------  -----          
 0   account_id                              4500 non-null   int64          
 1   account_district_id                     4500 non-null   int64          
 2   account_frequency                       4500 non-null   object         
 3   account_date                            4500 non-null   int64          
 4   account_year                            4500 non-null   int32          
 5   account_month                           4500 non-null   int32          
 6   account_day                             4500 non-null   int32          
 7   account_lor                             4500 non-null   int32          
 8   client_id                               4500 non-null   int64          
 9   client_district_id                      4

In [370]:
# RFM Table
# Merge recency dataframe with frequency dataframe
temp_df = recency_df.merge(freq_df,on='account_id')

# Merge monetary_df to temp_df 
rfm_df = temp_df.merge(monetary_df,on='account_id')
rfm_df.head(1)

,account_id,last_trans_date,trans_recency,trans_frequency,monetary
0,1,1998-12-31,0,239,375174.5


In [371]:
# RFM SCORE - ranking r,f,m  and normalize the values to calculate  the rfm score 


# Rank each metric Recency , Frequency & Monetary
rfm_df['r_rank'] = rfm_df['trans_recency'].rank( ascending=False)
rfm_df['f_rank'] = rfm_df['trans_frequency'].rank(ascending=True)
rfm_df['m_rank'] = rfm_df['monetary'].rank(ascending=True)


# normalize each rank with Max rank
rfm_df['r_rank_norm']=(rfm_df['r_rank']/rfm_df['r_rank'].max())*100
rfm_df['f_rank_norm']=(rfm_df['f_rank']/rfm_df['f_rank'].max())*100
rfm_df['m_rank_norm']=(rfm_df['m_rank']/rfm_df['m_rank'].max())*100


# RFM_Score = 0.15*R_rank_norm + 0.28*F_rank_norm + 0.57M_rank_norm**
rfm_df['rfm_Score']=0.15*rfm_df['r_rank_norm']+0.28*rfm_df['f_rank_norm']+0.57*rfm_df['m_rank_norm']
rfm_df=rfm_df.round(0)
rfm_df.head(2)

,account_id,last_trans_date,trans_recency,trans_frequency,monetary,r_rank,f_rank,m_rank,r_rank_norm,f_rank_norm,m_rank_norm,rfm_Score
0,1,1998-12-31,0,239,375174.0,2342.0,2574.0,946.0,100.0,57.0,21.0,43.0
1,2,1998-12-31,0,478,3151479.0,2342.0,4335.0,4020.0,100.0,96.0,89.0,93.0


In [372]:
#dropping varibales used to calculate rfm_score
rfm_df = rfm_df.drop(columns =['r_rank', 'f_rank','m_rank','r_rank_norm','f_rank_norm','m_rank_norm'])

In [373]:
# # Segment Accounts based on RFM score
# 0 - 30 - Low Value
# 30 - 70 - Gold
# 70 - 100 - Platinum
rfm_df["account_segment"]=np.where(rfm_df['rfm_Score'] > 70 ,"Platinum",
                                   (np.where(rfm_df['rfm_Score'] < 30 , "Low value" ,"Gold")))

In [374]:
rfm_df.head(2)

,account_id,last_trans_date,trans_recency,trans_frequency,monetary,rfm_Score,account_segment
0,1,1998-12-31,0,239,375174.0,43.0,Gold
1,2,1998-12-31,0,478,3151479.0,93.0,Platinum


In [375]:
#renaming the trans_type and trans_mode values 

trans['trans_type'] = trans['trans_type'].replace(['PRIJEM', 'VYDAJ'], ['Credit', 'Withdrawal'])

trans['trans_mode'] = trans['trans_mode'].replace(['VYBER KARTOU', 'VKLAD','PREVOD Z UCTU','VYBER','PREVOD NA UCET'], 
                                                  ['card_withdrawal', 'credit_cash','credit_otherbank','cash_withdrawal','transfer_otherbank'])


In [376]:
#Merging rfm_df to the trans96 
trans =  pd.merge(trans,rfm_df,'left',on = 'account_id')

In [377]:
trans.head(2)

,trans_id,account_id,trans_date,trans_type,trans_mode,trans_amount,trans_balance,trans_char,partner_bank,partner_account,trans_day,trans_month,trans_year,date,last_trans_date,trans_recency,trans_frequency,monetary,rfm_Score,account_segment
0,695247,2378,01-01-1993,Credit,credit_cash,700.0,700.0,NaN,NaN,NaN,1,1,1993,1993-01-01,1998-12-31,0,435,5315970.0,97.0,Platinum
1,171812,576,01-01-1993,Credit,credit_cash,900.0,900.0,NaN,NaN,NaN,1,1,1993,1993-01-01,1998-12-31,0,358,875619.0,66.0,Gold


In [378]:
trans["cash_credit"] = 0
trans["cash_withdrawal"] = 0


trans.loc[trans['trans_mode'] == 'cash_credit', 'cash_credit'] = trans.loc[trans['trans_mode'] == 'cash_credit','trans_amount']


trans.loc[trans['trans_mode'] == 'cash_withdrawal', 'cash_withdrawal'] = trans.loc[trans['trans_mode'] == 'cash_withdrawal','trans_amount']


trans_temp = trans.groupby(['account_id'], as_index = False).agg({'trans_amount' : 'sum','cash_credit' : 'sum',
                                                               'cash_withdrawal' : 'sum'})

trans_temp

,account_id,trans_amount,cash_credit,cash_withdrawal
0,1,375174.5,0.0,82774.0
1,2,3151479.3,0.0,1001191.0
2,3,295021.8,0.0,56949.8
3,4,350971.4,0.0,59813.4
4,5,166881.4,0.0,32044.4
...,...,...,...,...
4495,11333,3295795.5,0.0,1618737.9
4496,11349,3957372.2,0.0,1271219.0
4497,11359,2948081.4,0.0,1105500.6
4498,11362,1330010.5,0.0,336301.8


In [379]:
processed_table =  processed_table.merge(trans_temp, left_on = 'account_id', right_on = 'account_id', how = 'left')

In [380]:
processed_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4500 entries, 0 to 4499
Data columns (total 72 columns):
 #   Column                                  Non-Null Count  Dtype          
---  ------                                  --------------  -----          
 0   account_id                              4500 non-null   int64          
 1   account_district_id                     4500 non-null   int64          
 2   account_frequency                       4500 non-null   object         
 3   account_date                            4500 non-null   int64          
 4   account_year                            4500 non-null   int32          
 5   account_month                           4500 non-null   int32          
 6   account_day                             4500 non-null   int32          
 7   account_lor                             4500 non-null   int32          
 8   client_id                               4500 non-null   int64          
 9   client_district_id                      4

In [381]:
trans_temp.rename(columns={'trans_amount' : 'total_trans_amount'}, inplace = True)
trans_temp.rename(columns={'cash_credit' : 'total_cash_credit'}, inplace = True)
trans_temp.rename(columns={'cash_withdrawal' : 'total_cash_withdrawal'}, inplace = True)

In [382]:
#average balance and amount transacted per account 

avg_df = trans.groupby(by='account_id', as_index=False).agg({'trans_balance': 'mean','trans_amount':'mean' })

avg_df.columns = ['account_id','trans_avgbalance','trans_avgamount']


avg_df.trans_avgbalance = avg_df.trans_avgbalance.round()

avg_df.trans_avgamount = avg_df.trans_avgamount.round()


df = pd.merge(trans_temp,avg_df,'left',on = 'account_id')
df

,account_id,total_trans_amount,total_cash_credit,total_cash_withdrawal,trans_avgbalance,trans_avgamount
0,1,375174.5,0.0,82774.0,15994.0,1570.0
1,2,3151479.3,0.0,1001191.0,36541.0,6593.0
2,3,295021.8,0.0,56949.8,26974.0,2522.0
3,4,350971.4,0.0,59813.4,21753.0,1887.0
4,5,166881.4,0.0,32044.4,25090.0,1987.0
...,...,...,...,...,...,...
4495,11333,3295795.5,0.0,1618737.9,46304.0,8956.0
4496,11349,3957372.2,0.0,1271219.0,48769.0,13018.0
4497,11359,2948081.4,0.0,1105500.6,36106.0,7799.0
4498,11362,1330010.5,0.0,336301.8,30470.0,3866.0


In [383]:
processed_table =  processed_table.merge(df, left_on = 'account_id', right_on = 'account_id', how = 'left')


In [384]:
processed_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4500 entries, 0 to 4499
Data columns (total 77 columns):
 #   Column                                  Non-Null Count  Dtype          
---  ------                                  --------------  -----          
 0   account_id                              4500 non-null   int64          
 1   account_district_id                     4500 non-null   int64          
 2   account_frequency                       4500 non-null   object         
 3   account_date                            4500 non-null   int64          
 4   account_year                            4500 non-null   int32          
 5   account_month                           4500 non-null   int32          
 6   account_day                             4500 non-null   int32          
 7   account_lor                             4500 non-null   int32          
 8   client_id                               4500 non-null   int64          
 9   client_district_id                      4

In [385]:
trans.rename(columns={'account_id' : 'trans_account_id'}, inplace = True)
trans.rename(columns={'trans_char' : 'trans_k_symbol'}, inplace = True)
trans.rename(columns={'trans_mode' : 'trans_operation'}, inplace = True)

In [390]:
trans_k_symbol_agg = trans.loc[trans['trans_year'] == 1996].groupby(['trans_account_id', 'trans_k_symbol'])['trans_amount'].agg(['count','sum']).unstack('trans_k_symbol')
processed_table =  processed_table.merge(trans_k_symbol_amount, left_on = 'account_id', right_on = 'trans_account_id', how = 'left')

trans_operation_agg = trans.loc[trans['trans_year'] == 1996].groupby(['trans_account_id', 'trans_operation'])['trans_amount'].agg(['count','sum']).unstack('trans_operation')
processed_table =  processed_table.merge(trans_operation_agg, left_on = 'account_id', right_on = 'trans_account_id', how = 'left')

trans_type_agg = trans.loc[trans['trans_year'] == 1996].groupby(['trans_account_id', 'trans_type'])['trans_amount'].agg(['count','sum']).unstack('trans_type')
processed_table =  processed_table.merge(trans_type_agg, left_on = 'account_id', right_on = 'trans_account_id', how = 'left')


C:\tools\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
C:\tools\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
C:\tools\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [391]:
processed_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4500 entries, 0 to 4499
Columns: 141 entries, account_id to ('sum', 'Withdrawal')
dtypes: float64(103), int32(12), int64(17), object(8), timedelta64[ns](1)
memory usage: 4.7+ MB
